# 簡単なモデルをつくってみよう（３）

- 細胞外の物質を定義する。

## E-Cell3を使うための準備

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

## %install_ext https://raw.githubusercontent.com/naito/ecell3-ipython/master/extensions/ecell3.py
## %load_ext ecell3

In [ ]:
import biosim_course

# モデル３

- 外部からの物質の流入・流出を表現する。
- 外部の物質は、増減しないことにする。
  - `Variable` の属性 `Fixed` を**真**（1）に設定する。

![モデル３概要](./model3.png)

# このモデルに含まれる `Entity`

## `System`
- ルートシステム（細胞外の空間） FullID = `System::/'
- 細胞 FullID = `System:/:Cell'

## `Variable`
- ルートシステム（細胞外の空間）
  - 物質１ (`S1`)　 FullID = `Varible:/:S1'
  - 物質２ (`S2`)　 FullID = `Varible:/:S2'

- 細胞
  - 物質１ (`S1`)　 FullID = `Varible:/Cell:S1'
  - 物質２ (`S2`)　 FullID = `Varible:/Cell:S2'

## `Process`
- 細胞
  - 酵素１ (`E1`)　 FullID = `Process:/Cell:E1'
  - 酵素２ (`E2`)　 FullID = `Process:/Cell:E2'
  - 輸送体１ (`T1`)　 FullID = `Process:/Cell:T1'
  - 輸送体２ (`T2`)　 FullID = `Process:/Cell:T2'


## EM の記述

- ルートシステム（細胞外）の `Variable` の属性 `Fixed` を**真**（1）に設定する。
- 輸送体 `T1`、`T2` に関連する２つの `Variable` のID（名前）は同一なので、`VariableReferenceList` での参照名は異なる名前をつける。
  - ここでは、外（ex）、内（in）を表す文字列を参照名にしてみた。

In [ ]:
EM = '''
Stepper ODEStepper( ODE ){}

System System( / )
{
    StepperID    ODE;

    Variable Variable( SIZE )
    {
        Value    1;
    }

    Variable Variable( S1 )
    {
        Value    10000;
        Fixed    1;
    }

    Variable Variable( S2 )
    {
        Value    10000;
        Fixed    1;
    }

}

System System( /Cell )
{
    StepperID    ODE;

Variable Variable( SIZE )
    {
        Value    1;
    }

    Variable Variable( S1 )
    {
        Value    10000;
    }

    Variable Variable( S2 )
    {
        Value    10000;
    }

    Process MassActionFluxProcess( E1 )
    {
        k    0.1;
        VariableReferenceList
            [ S1  Variable:/Cell:S1 -1 ]
            [ S2  Variable:/Cell:S2  1 ];
    }

    Process MassActionFluxProcess( E2 )
    {
        k    0.2;
        VariableReferenceList
            [ S2  Variable:/Cell:S2 -1 ]
            [ S1  Variable:/Cell:S1  1 ];
    }

    Process MassActionFluxProcess( T1 )
    {
        k    0.2;
        VariableReferenceList
            [ ex  Variable:/:S1     -1 ]
            [ in  Variable:/Cell:S1  1 ];
    }

    Process MassActionFluxProcess( T2 )
    {
        k    0.1;
        VariableReferenceList
            [ ex  Variable:/:S2      1 ]
            [ in  Variable:/Cell:S2 -1 ];
    }

}
'''

# モデルを走らせてみる

In [ ]:
setModel(EM, "model")

print 't = {}'.format( getCurrentTime() )

## Loggerをつくる

In [ ]:
target_SystemPath_list = ( '/', '/Cell', )
Target_Properties = { 'Variable': ['Value', 'Velocity'], }
Logger_dict = {}

for target_SystemPath in target_SystemPath_list:
    for E_type, Properties in Target_Properties.items():
        for E in getEntityList( E_type, target_SystemPath ):
            for p in Properties:
                FullPN = ':'.join( ( E_type, target_SystemPath, E, p ) )
                Logger_dict[ FullPN ] = createLoggerStub( FullPN )
                Logger_dict[ FullPN ].create()

## シミュレーションを実行する

In [ ]:
step_width = 60.0  # min

run( step_width )
print 't = {}'.format( getCurrentTime() )

## グラフを描いてみる

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(), getCurrentTime() / 100 ) )[ :, :2 ]

FullPNs_for_plot = [ 
    'Variable:/Cell:S1:Value',
    'Variable:/Cell:S2:Value',
    ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

### 定常状態に達していないので、もう少しシミュレーションしてみる。

In [ ]:
run( step_width * 9 )
print 't = {}'.format( getCurrentTime() )

In [ ]:
Data_dict = {}
for FullPN, Logger in Logger_dict.items():
    Data_dict[ FullPN ] = np.array( Logger.getData( 0, getCurrentTime(), getCurrentTime() / 100 ) )[ :, :2 ]

FullPNs_for_plot = [ 
    'Variable:/Cell:S1:Value',
    'Variable:/Cell:S2:Value',
    ]

plt.figure()
for FullPN, d in Data_dict.items():
    if FullPN in FullPNs_for_plot:
        plt.plot( d[ :, 0 ], d[ :, 1 ], label = FullPN.split(':')[ 2 ] )

plt.legend( loc = 'upper right' )

# 演習（１）
- このモデルの定常状態はどのように決まるだろうか。
  - 速度定数などモデル内の値をいろいろ変えて試してみてもよい。
  - 数学的に解いてもよい。
  - いろいろ試行錯誤してみよう。